In [1]:
import mlflow
from mlflow.tracking import MlflowClient
import csv

mlflow_uri = "http://cimmerian.win:5000"
mlflow.set_tracking_uri(uri=mlflow_uri)

In [2]:
client = MlflowClient()

In [3]:
def get_history_value(client, run_id, metric, step):
    return next(filter(lambda x: x.step == step, client.get_metric_history(run_id, metric)))

In [4]:
def caputre_metrics_into_csv(filename, run_id):
    val_class_size = [116, 263, 237, 248, 251, 169]
    val_sum = [0, 0, 0, 0]
    classes = [
        'pants-fire',
        'false',
        'barely-true',
        'half-true',
        'mostly-true',
        'true'
    ]

    fn = lambda x: f'{round(x, 2):.02f}'
    
    step = sorted([(x.step, x.value) for x in client.get_metric_history(run_id, 'val_acc')], key=lambda x: -x[1])[0][0]
    with open(filename, 'w', encoding='utf-8', newline='\n') as f:
        writer = csv.writer(f)
        i = 0
        writer.writerow(['Klasa', 'Rozmiar klasy', 'Precyzja', 'Pełność', 'F1'])
        for _class in classes:
            precision_label = 'val_precision_' + _class
            recall_label = 'val_recall_' + _class
            f1_label = 'val_f1_' + _class
            precision = get_history_value(client, run_id, precision_label, step).value
            recall = get_history_value(client, run_id, recall_label, step).value
            f1 = get_history_value(client, run_id, f1_label, step).value
            row = [
                classes[i],
                val_class_size[i],
                fn(precision),
                fn(recall),
                fn(f1)
            ]
            val_sum = [
                val_sum[0] + row[1],
                val_sum[1] + precision,
                val_sum[2] + recall,
                val_sum[3] + f1
            ]
            writer.writerow(row)
            i += 1
        val_sum = [
            val_sum[0],
            fn(val_sum[1] / len(classes)),
            fn(val_sum[2] / len(classes)),
            fn(val_sum[3] / len(classes))
        ]
        writer.writerow(['suma/średnia'] + val_sum)

In [5]:
caputre_metrics_into_csv('multiple_metrics.csv', '3c0265c507f1470dba04ad08df168f72')
caputre_metrics_into_csv('single_metrics.csv', 'b8dbfe59dc9b4073a5c17d80af6812af')
caputre_metrics_into_csv('single_with_dropout_metrics.csv', '9ffc927ca40c40b39036d5b3930ac70c')
caputre_metrics_into_csv('finetuned_metrics.csv', 'c5a7978dca644e5189130554f97ba734')

In [9]:
a = max([x.value for x in client.get_metric_history('b8dbfe59dc9b4073a5c17d80af6812af', 'val_loss')])
a

1.7561219476518177

In [10]:
b = min([x.value for x in client.get_metric_history('b8dbfe59dc9b4073a5c17d80af6812af', 'val_loss')])
b

1.7003307796659923

In [11]:
a - b

0.055791167985825396

In [12]:
a = max([x.value for x in client.get_metric_history('9ffc927ca40c40b39036d5b3930ac70c', 'val_loss')])
a

1.760870201247079

In [13]:
b = min([x.value for x in client.get_metric_history('9ffc927ca40c40b39036d5b3930ac70c', 'val_loss')])
b

1.7229948157355899

In [14]:
a - b

0.03787538551148906